# Clustering collums and rows

The idea is to cluster collumns and rows

Here we have the objective function 



In [ ]:
import numpy as np
import tvsclib.utils as utils
import Split
import matplotlib.pyplot as plt

import torchvision.models as models
import torch

In [ ]:
def get_mobilenet_target_mats():
    target_mats = []
    # Load the model
    model = models.mobilenet_v2(pretrained=True)
    # Put moel into eval mode
    model.eval()
    for layer in model.classifier:
        if isinstance(layer, torch.nn.Linear):
            # Obtain the weights of this layer
            weights = layer.weight.detach().numpy()
            target_mats.append(weights)
    return target_mats

In [ ]:
T = np.random.rand(32,32)
#T = mats = get_mobilenet_target_mats()[0]
sys = Split.initial_mixed(T)
utils.show_system(sys,mark_D=False)
utils.check_dims(sys)

In [ ]:
#get matricies
k = 0
stage_c=sys.causal_system.stages[k]
stage_a=sys.anticausal_system.stages[k]
Ac = stage_c.A_matrix
Bc = stage_c.B_matrix
Cc = stage_c.C_matrix

Aa = stage_a.A_matrix
Ba = stage_a.B_matrix
Ca = stage_a.C_matrix

D = stage_c.D_matrix

#dims of states

(d_out_c,d_in_c) = Ac.shape

(d_out_a,d_in_a) = Aa.shape

X = np.block([[np.zeros((d_out_a,d_in_c)),Ba,Aa ],
              [Cc,D,Ca],
              [Ac,Bc,np.zeros((d_out_c,d_in_a))]
    
])

plt.matshow(X)

s_c = np.zeros(X.shape[1],dtype=bool)
s_c[:d_in_c+D.shape[1]//2]=1
s_r = np.zeros(X.shape[0],dtype=bool)
s_r[d_out_a+D.shape[0]//2:]=1

In [ ]:
X_l = X[:,s_c]
X_r = X[:,~s_c]

X_l = X_l/np.linalg.norm(X_l,axis=1).reshape(-1,1)
X_r = X_r/np.linalg.norm(X_r,axis=1).reshape(-1,1)

X_t = X[~s_r]
X_b = X[s_r]

X_t = X_t/np.linalg.norm(X_t,axis=0).reshape(1,-1)
X_b = X_b/np.linalg.norm(X_b,axis=0).reshape(1,-1)

In [ ]:
X_ = X_b #top or bottom
print(np.diag(X_.T@X_))
plt.matshow(X_.T@X_)

In [ ]:
X_ = X_r  #left or right
print(np.diag(abs(X_@X_.T)))
plt.matshow(X_@X_.T)

In [ ]:
plt.matshow(1-abs(X_@X_.T))

In [ ]:
Adj = 1-abs(X_b.T@X_b)
np.sum(Adj[:,s_c],axis=1)

In [ ]:
Adj = 1-abs(X_t.T@X_t)
np.sum(Adj[:,~s_c],axis=1)

In [ ]:
Adj = 1-abs(X_l@X_l.T)
np.sum(Adj[:,s_r],axis=1)

In [ ]:
Adj = 1-abs(X_r@X_r.T)
np.sum(Adj[:,~s_r],axis=1)

In [ ]:
def f(X,s_col,s_row):
    return np.linalg.norm(X[s_row][:,s_col],'nuc')+np.linalg.norm(X[~s_row][:,~s_col],'nuc')

def compute_sigmasf(X,s_col,s_row):
    return np.linalg.svd(X[s_row][:,s_col],compute_uv=False), np.linalg.svd(X[~s_row][:,~s_col],compute_uv=False)

def show_matrices(X,s_col,s_row):
    plt.matshow(X[s_row][:,s_col],fignum=1)

    plt.matshow(X[~s_row][:,~s_col],fignum=2)

f_reg_row = lambda x: -gamma*x**3
f_reg_col = lambda x: -gamma*x**3

gamma = 1e8 #maxbee have two different regularizations for rows and collumns

In [ ]:
def segment_matrix(stage_causal,stage_anticausal,N=50):
    
    Ac = stage_causal.A_matrix
    Bc = stage_causal.B_matrix
    Cc = stage_causal.C_matrix

    Aa = stage_anticausal.A_matrix
    Ba = stage_anticausal.B_matrix
    Ca = stage_anticausal.C_matrix

    D = stage_c.D_matrix
    
    #regularization vector
    v_reg_col = f_reg_col(np.linspace(-1,1,D.shape[1]))
    v_reg_row = f_reg_row(np.linspace(-1,1,D.shape[0]))

    #dims of states
    (d_out_c,d_in_c) = Ac.shape
    (d_out_a,d_in_a) = Aa.shape

    #setup matrix
    X = np.block([[np.zeros((d_out_a,d_in_c)),Ba,Aa ],
                  [Cc,D,Ca],
                  [Ac,Bc,np.zeros((d_out_c,d_in_a))]
    
    ])

    #initialize segmentation
    s_c = np.zeros(X.shape[1],dtype=bool)
    s_c[:d_in_c+D.shape[1]//2]=1
    s_r = np.zeros(X.shape[0],dtype=bool)
    s_r[d_out_a+D.shape[0]//2:]=1
    
    
    fs = np.zeros(N+1)
    s_cols=np.zeros((N+1,X.shape[1]),dtype=bool) 
    s_rows=np.zeros((N+1,X.shape[0]),dtype=bool) 
    
    s_cols[0]=s_c
    s_rows[0]=s_r
    fs[0]=f(X,s_c,s_r)
    
    for n in range(N):
        #normlize
        X_l = X[:,s_c]
        X_r = X[:,~s_c]

        X_l = X_l/np.linalg.norm(X_l,axis=1).reshape(-1,1)
        X_r = X_r/np.linalg.norm(X_r,axis=1).reshape(-1,1)

        X_t = X[~s_r]
        X_b = X[s_r]

        X_t = X_t/np.linalg.norm(X_t,axis=0).reshape(1,-1)
        X_b = X_b/np.linalg.norm(X_b,axis=0).reshape(1,-1)
    
        #compute cost matricies and cost vectors 
        #columns
        Adj_b = 1-abs(X_b.T@X_b)
        Adj_t = 1-abs(X_t.T@X_t)
        f_col = np.sum(Adj_b[:,s_c],axis=1) - np.sum(Adj_t[:,~s_c],axis=1)
        
        Adj_l = 1-abs(X_l@X_l.T)
        Adj_r = 1-abs(X_r@X_r.T)
        f_row = np.sum(Adj_l[:,s_r],axis=1) - np.sum(Adj_r[:,~s_r],axis=1)
        

        
        #try to minimize -> choose smaller one
        
        ord_c = d_in_c+np.argsort(f_col[d_in_c:X.shape[1]-d_in_a])
        ord_r = d_out_a+np.argsort(f_row[d_out_a:X.shape[0]-d_out_c])
        s_c[d_in_c:X.shape[1]-d_in_a]=0
        s_r[d_out_a:X.shape[0]-d_out_c]=0
        v_c =ord_c[f_col[ord_c]<v_reg_col]
        v_r =ord_r[f_col[ord_r]<v_reg_row]
        s_c[v_c]=1
        s_r[v_r]=1
        
        #print(f_col)
        #print(f_row)
        
        #s_c[:]=0
        #s_c[f_col<0]=1
        #s_r[:]=0
        #s_r[f_row<0]=1
        
        
        fs[n+1] = f(X,s_c,s_r)
        s_cols[n+1]=s_c
        s_rows[n+1]=s_r

    return fs,s_cols,s_rows,X
        


In [ ]:
sys = Split.initial_mixed(T)
stage_c=sys.causal_system.stages[k]
stage_a=sys.anticausal_system.stages[k]
fs,s_cols,s_rows,X = segment_matrix(stage_c,stage_a,N=10)

In [ ]:
s_start = np.linalg.svd(X[s_rows[0]][:,s_cols[0]],compute_uv=False)
s_end = np.linalg.svd(X[s_rows[-1]][:,s_cols[-1]],compute_uv=False)

plt.plot(s_start,label="start")
plt.plot(s_end,label="end")
plt.legend()

In [ ]:
s_start_c,s_start_a = compute_sigmasf(X,s_cols[0],s_rows[0])
s_end_c,s_end_a = compute_sigmasf(X,s_cols[-1],s_rows[-1])

plt.subplot(1,2,1)
plt.plot(s_start_c,label="start")
plt.plot(s_end_c,label="end")
plt.legend()
plt.subplot(1,2,2)
plt.plot(s_start_a,label="start")
plt.plot(s_end_a,label="end")

In [ ]:
plt.spy(s_cols)

In [ ]:
plt.spy(s_rows)

In [ ]:
plt.plot(fs)

In [ ]:
show_matrices(X,s_rows[-1],s_cols[-1])

In [ ]:
plt.matshow(X[s_r][:,s_c])

In [ ]:
plt.matshow(X[~s_r][:,~s_c])